In [2]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tqdm import tqdm

# Define the path to the directory
directory_path = r"G:\datasets\Underwater_Image\WHOI\archive\dataset_pm\training"

# Initialize empty lists for images and labels
X_train = []
Y_train = []
class_label = 0

# Loop through subdirectories (classes)
for class_folder in tqdm(sorted(os.listdir(directory_path))):
    class_path = os.path.join(directory_path, class_folder)
    
    for image_file in os.listdir(class_path):
        if image_file.endswith('.png'):
            image_path = os.path.join(class_path, image_file)
            
            # Load image, convert to RGB and resize
            img = Image.open(image_path).convert('RGB')
            img = img.resize((32, 32))
            img_array = np.array(img)
            
            # Append image and label to lists
            X_train.append(img_array)
            Y_train.append(class_label)
            
    class_label = class_label + 1

# Convert lists to numpy arrays
X_train = np.array(X_train)
Y_train = np.array(Y_train)

print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')

100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [10:01<00:00, 31.67s/it]

X_train shape: (2872, 32, 32, 3)
Y_train shape: (2872,)


In [1]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tqdm import tqdm

# Define the path to the directory
directory_path = r"E:\imbcifar\train"

# Initialize empty lists for images and labels
X_train = []
Y_train = []
class_label = 0

# Loop through subdirectories (classes)
for class_folder in tqdm(sorted(os.listdir(directory_path))):
    class_path = os.path.join(directory_path, class_folder)
    
    for image_file in os.listdir(class_path):
        if image_file.endswith('.png'):
            image_path = os.path.join(class_path, image_file)
            
            # Load image, convert to RGB and resize
            img = Image.open(image_path).convert('RGB')
            img = img.resize((32, 32))
            img_array = np.array(img)
            
            # Append image and label to lists
            X_train.append(img_array)
            Y_train.append(class_label)
            
    class_label = class_label + 1

# Convert lists to numpy arrays
X_train = np.array(X_train)
Y_train = np.array(Y_train)

print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:30<00:00,  9.02s/it]

X_train shape: (29009, 32, 32, 3)
Y_train shape: (29009,)


In [1]:
import dask.dataframe as dd
import numpy as np
from tqdm import tqdm

# Define the paths to the CSV files
file_paths = [
    r"E:\imagenet\lbpfeature1.csv",
    r"E:\imagenet\lbpfeature2.csv",
    r"E:\imagenet\lbpfeature3.csv",
    r"E:\imagenet\lbpfeature4.csv"
]

# Load the CSV files into Dask DataFrames and immediately drop rows with NaN values
dfs = [dd.read_csv(file_path).dropna() for file_path in file_paths]

# Function to get random indexes and retrieve the full rows
def get_random_rows(df, n=250000):
    # Convert the Dask DataFrame to a Pandas DataFrame after dropping NaNs
    pandas_df = df.compute()
    
    # Get the total number of rows in the DataFrame
    total_rows = len(pandas_df)
    
    # If there are fewer rows than n, use the total number of rows instead
    n = min(total_rows, n)
    
    # Generate random indexes
    random_indexes = np.random.choice(total_rows, n, replace=False)
    
    # Retrieve the full rows for these random indexes and split on tabs
    random_rows = [
        [item for sublist in (str(cell).split('\t') for cell in row) for item in sublist]
        for row in pandas_df.iloc[random_indexes].values
    ]
    
    return random_rows

# Initialize an empty list to store all rows
all_random_rows = []

# Get the rows from each DataFrame
for df in tqdm(dfs, desc="Processing DataFrames"):
    rows = get_random_rows(df)
    all_random_rows.extend(rows)

# Print the total number of rows collected
print(len(all_random_rows))


Processing DataFrames: 100%|██████████████████████████████████████████| 4/4 [00:56<00:00, 14.24s/it]

994940


In [2]:
# Remove the first and last items from each element in all_random_rows
new_all_random_rows = [row[1:-1] for row in all_random_rows]
print(np.shape(new_all_random_rows))

(994940, 256)


In [3]:
import pickle
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans

# Assuming new_all_random_rows has been populated as described

# Convert new_all_random_rows to a NumPy array
data = np.array(new_all_random_rows)

batch_size = 25000
max_iter = 100

# Apply K-means clustering
kmeans = MiniBatchKMeans(n_clusters=1000, batch_size=batch_size, max_iter=max_iter, random_state=42)
kmeans.fit(data)

# Store the cluster centers
centers = kmeans.cluster_centers_

# Print the shape of the cluster centers
print(f"Shape of the cluster centers: {centers.shape}")

# Save the cluster centers to a pickle file
with open('cluster_centers.pkl', 'wb') as file:
    pickle.dump(centers, file)

print("Cluster centers have been saved to 'cluster_centers.pkl'.")


C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\sklearn\cluster\_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


Shape of the cluster centers: (1000, 256)
Cluster centers have been saved to 'cluster_centers.pkl'.


In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


In [5]:
import numpy as np
from skimage.color import rgb2gray
from skimage.feature import local_binary_pattern
from skimage.transform import resize
from sklearn.cluster import MiniBatchKMeans
from tqdm import tqdm

# LBP parameters
radius = 3
n_points = 8 * radius  # Total number of points
METHOD = 'uniform'
num_bins = 256  # Number of bins for the histogram

def extract_features(image, target_size=(32, 32)):
    # Process the image
    image_resized = resize(image, target_size, anti_aliasing=True)
    image_gray = rgb2gray(image_resized)
    lbp = local_binary_pattern(image_gray, n_points, radius, METHOD)
    hist, _ = np.histogram(lbp, density=True, bins=num_bins, range=(0, num_bins))
    return hist

def process_images(images):
    features = []
    
    for image in tqdm(images):
        try:
            hist = extract_features(image)
            features.append(hist)
        except Exception as e:
            print(f"Failed to process an image: {e}")
    return features

# Example usage, assuming x_train is already loaded with numpy arrays of images
features = process_images(x_train)

# Clustering with MiniBatchKMeans
n_clusters = 400
batch_size = 100
max_iter = 100

kmeans = MiniBatchKMeans(n_clusters=n_clusters, batch_size=batch_size, max_iter=max_iter)
kmeans.fit(np.array(features))
ct = kmeans.cluster_centers_.tolist()


  0%|                                                                     | 0/50000 [00:00<?, ?it/s]C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\skimage\feature\texture.py:353: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(
100%|████████████████████████████████████████████████████████| 50000/50000 [01:21<00:00, 612.55it/s]
C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\sklearn\cluster\_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


In [11]:
np.shape(ct)

(400, 256)

In [6]:
import numpy as np
from tqdm import tqdm

# Initialize a list to store the distances
tot_dist = []

# Calculate L2 distances
for i in tqdm(range(len(ct))):
    distances = []
    for j in range(len(centers)):
        distance = np.linalg.norm(ct[i] - centers[j])
        distances.append(distance)
    tot_dist.append(distances)

# Convert the list to a numpy array
tot_dist = np.array(tot_dist)

# Print the shape of the resulting array
print(f"Shape of the distance array: {tot_dist.shape}")

100%|█████████████████████████████████████████████████████████████| 400/400 [00:08<00:00, 48.03it/s]

Shape of the distance array: (400, 1000)


In [7]:
from ortools.linear_solver import pywraplp
dist_list =  np.transpose(tot_dist, (1, 0))
from tqdm import tqdm

costs = dist_list

num_workers = len(costs)
num_tasks = len(costs[0])
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver("SCIP")

# x[i, j] is an array of 0-1 variables, which will be 1
# if worker i is assigned to task j.
x = {}
for i in range(num_workers):
    for j in range(num_tasks):
        x[i, j] = solver.IntVar(0, 1, "")
        
# Each worker is assigned to at most 1 task.
for i in range(num_workers):
    solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

# Each task is assigned to exactly one worker.
for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)
    
objective_terms = []
for i in tqdm(range(num_workers)):
    for j in range(num_tasks):
        objective_terms.append(costs[i][j] * x[i, j])
solver.Minimize(solver.Sum(objective_terms))

status = solver.Solve()

sol_indexes = []
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    for i in range(num_workers):
        for j in range(num_tasks):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.1:
                print(f"Cluster {j} assigned to Class {i}." + f" Cost: {costs[i][j]}")
                sol_indexes.append(i)
else:
    print("No solution found.")

100%|███████████████████████████████████████████████████████████| 1000/1000 [00:22<00:00, 44.05it/s]


Total cost = 1364.0356541084543

Cluster 280 assigned to Class 0. Cost: 3.8959321753766325
Cluster 2 assigned to Class 3. Cost: 2.986787980180775
Cluster 93 assigned to Class 5. Cost: 3.4249302979573133
Cluster 160 assigned to Class 11. Cost: 3.4083469260901853
Cluster 307 assigned to Class 12. Cost: 3.3835023439833534
Cluster 208 assigned to Class 15. Cost: 2.7702192819836338
Cluster 15 assigned to Class 18. Cost: 3.48376051501223
Cluster 156 assigned to Class 20. Cost: 3.371691266129736
Cluster 252 assigned to Class 22. Cost: 3.4763524878423926
Cluster 251 assigned to Class 23. Cost: 2.9865940658701566
Cluster 327 assigned to Class 24. Cost: 3.5321905771257915
Cluster 92 assigned to Class 29. Cost: 3.5080448162892486
Cluster 66 assigned to Class 30. Cost: 3.433456745300317
Cluster 261 assigned to Class 32. Cost: 3.5223143456787374
Cluster 178 assigned to Class 34. Cost: 2.9672819396136645
Cluster 88 assigned to Class 38. Cost: 2.926131048399946
Cluster 162 assigned to Class 39. Cost:

In [8]:
len(sol_indexes)

400

In [9]:
with open('cvfinal.pkl', 'wb') as file:
    pickle.dump(sol_indexes, file)

print("Indexes have been saved to 'cvfinal.pkl'.")

Indexes have been saved to 'cvfinal.pkl'.


Time taken: 1.6200830936431885 seconds


In [42]:
import dask.dataframe as dd
import numpy as np
from scipy.spatial.distance import cdist
from dask.distributed import Client
import pickle
import time
from tqdm import tqdm
import dask

# Initialize Dask client
client = Client()

def compute_closest_indexes(chunk, centers, index_set):
    result_list = []
    print("Centers shape:", np.shape(centers))
    # Convert the DataFrame to a list of lists and handle tab-separated values
    chunk_list = [row[0].split('\t') for row in chunk.values.tolist()]
    print("Chunk DataFrame as list after splitting tabs:\n", chunk_list)

    for idx, row in enumerate(chunk_list):
        # Remove the first and last element from each row and convert the rest to float
        features = np.array(row[1:-1], dtype=float).reshape(1, -1)  # Skipping the first element (file path) and the last element
        print("Processed features array:", features)  # Print features for debugging

        # Compute the Euclidean distance to each center
        distances = cdist(features, centers, metric='euclidean')
        closest_center_idx = np.argmin(distances)
        
        # Append results if closest center is in the index set
        if closest_center_idx in index_set:
            result_list.append((closest_center_idx, row[1]))  # row[1] should still be the ID or similar

    return result_list

# Function to process elements of all DataFrames
def process_elements(dfs, centers, index_set, max_len_per_index=500):
    result_list = []
    index_count = {idx: 0 for idx in index_set}
    delayed_results = []
    print(len(result_list))
    for df in tqdm(dfs, desc="Processing DataFrames"):
        # Iterate over chunks of the DataFrame
        for chunk in tqdm(df.to_delayed(), desc="Processing chunks", leave=False):
            # Process each chunk in parallel
            delayed_result = dask.delayed(compute_closest_indexes)(chunk, centers, index_set)
            delayed_results.append(delayed_result)

        # Compute and collect results
        if delayed_results:
            chunk_results = dask.compute(*delayed_results)
            for chunk_result in chunk_results:
                for closest_center_idx, row_1 in chunk_result:
                    if index_count[closest_center_idx] < max_len_per_index:
                        result_list.append(row_1)
                        index_count[closest_center_idx] += 1
                        if index_count[closest_center_idx] >= max_len_per_index:
                            index_set.remove(closest_center_idx)
                        if all(count >= max_len_per_index for count in index_count.values()):
                            return result_list

    return result_list

# Measure the execution time
start_time = time.time()

# Load the pickle files
with open("D:/ML_notebooks/cvfinal.pkl", 'rb') as file:
    index = pickle.load(file)
with open("D:/ML_notebooks/cluster_centers.pkl", 'rb') as file:
    centers = pickle.load(file)

# Convert index to a set for faster lookup
index_set = set(index)

# Define the paths to the CSV files
file_paths = [
    r"E:\imagenet\lbpfeature1.csv",
    r"E:\imagenet\lbpfeature2.csv",
    r"E:\imagenet\lbpfeature3.csv",
    r"E:\imagenet\lbpfeature4.csv"
]

# Load the CSV files into Dask DataFrames
dfs = [dd.read_csv(file_path) for file_path in file_paths]

# Call the function to process elements of each DataFrame
result_list = process_elements(dfs, centers, index_set)

# Save the result_list in a pickle file
#with open("cifar_images.pkl", "wb") as file:
#    pickle.dump(result_list, file)

end_time = time.time()

# Print the execution time
print(f"Execution time: {end_time - start_time} seconds")


C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 58656 instead
  warnings.warn(
C:\Users\shaif\anaconda3\envs\tens\lib\contextlib.py:120: UserWarning: Creating scratch directories is taking a surprisingly long time. (5.65s) This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  next(self.gen)


0


Processing DataFrames: 100%|█████████████████████████████████████████| 4/4 [56:57<00:00, 854.41s/it]

Execution time: 3418.967207431793 seconds


In [41]:
print(len(result_list))

197367


In [37]:
def compute_closest_indexes(chunk, centers, index_set):
    result_list = []
    print("Centers shape:", np.shape(centers))
    # Convert the DataFrame to a list of lists and handle tab-separated values
    chunk_list = [row[0].split('\t') for row in chunk.values.tolist()]
    print("Chunk DataFrame as list after splitting tabs:\n", chunk_list)

    for idx, row in enumerate(chunk_list):
        # Remove the first and last element from each row and convert the rest to float
        features = np.array(row[1:-1], dtype=float).reshape(1, -1)  # Skipping the first element (file path) and the last element
        print("Processed features array:", features)  # Print features for debugging

        # Compute the Euclidean distance to each center
        distances = cdist(features, centers, metric='euclidean')
        closest_center_idx = np.argmin(distances)
        
        # Append results if closest center is in the index set
        if closest_center_idx in index_set:
            result_list.append((closest_center_idx, row[1]))  # row[1] should still be the ID or similar

    return result_list

# Assuming 'df', 'centers', and 'index_set' are defined and initialized appropriately
chunk = df.iloc[1:10]  # Get a subset of the DataFrame
results = compute_closest_indexes(chunk, centers, index_set)
print("Results:", results)


Centers shape: (1000, 256)
Chunk DataFrame as list after splitting tabs:
 [['D:/data/imagenet\\n01440764\\n01440764_10027.JPEG', '0.5673076923076923', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.21153846153846154', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.18269230769230768', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.22115384615384615', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.25961538461538464', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.25', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.2980769230769231', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.33653846153846156', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.4326923076923077', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.5096153846153846', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.4423076923076923', '0.0', '0.0', '0.0', '

Processed features array: [[0.60576923 0.         0.         0.         0.         0.
  0.         0.         0.         0.30769231 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.26923077 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.18269231
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.22115385 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.23076923 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.28846154
  0.         0.         0.         0.         0.         0.
  0.         0.         0.48076923 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.40384615 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.40384615 0.
  0.         0.         0.         0.         0.         0